In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
petr = pd.read_excel('petr4.xlsx')

In [38]:
petr.head()

,close
date,
2000-01-03,5.875
2000-01-04,5.550
2000-01-05,5.494
2000-01-06,5.475
2000-01-07,5.500


In [39]:
from statsmodels.tsa.stattools import adfuller
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [40]:
adf_test(petr['close'])

Augmented Dickey-Fuller Test: 
ADF test statistic        -2.206589
p-value                    0.203863
# lags used               32.000000
# observations          5644.000000
critical value (1%)       -3.431509
critical value (5%)       -2.862052
critical value (10%)      -2.567043
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary


In [41]:
df_difference = petr.diff()

In [42]:
df_difference

,close
date,
2000-01-03,NaN
2000-01-04,-0.325000
2000-01-05,-0.056000
2000-01-06,-0.019000
2000-01-07,0.025000
...,...
2022-07-29,1.860001
2022-08-01,-0.470001
2022-08-02,0.150002


In [43]:
adf_test(df_difference['close'])

Augmented Dickey-Fuller Test: 
ADF test statistic     -1.223349e+01
p-value                 1.040742e-22
# lags used             3.100000e+01
# observations          5.644000e+03
critical value (1%)    -3.431509e+00
critical value (5%)    -2.862052e+00
critical value (10%)   -2.567043e+00
Strong evidence against the null hypothesis
Reject the null hypothesis
Data has no unit root and is stationary


In [44]:
test_obs = 12
train = df_difference[:-test_obs]
test = df_difference[-test_obs:]

In [45]:
import numpy as np

import pandas

import statsmodels.api as sm

from statsmodels.tsa.api import VAR

In [48]:
for i in [1,2,3,4,5,6,7,8,9,10]:
    model = VAR(train)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()

/home/matheus/miniconda3/envs/econometria/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: Only gave one variable to VAR

In [36]:
np.asarray(df_difference)

array([[NaT, nan],
       [Timedelta('1 days 00:00:00'), -0.3249998092651367],
       [Timedelta('1 days 00:00:00'), -0.0560002326965332],
       ...,
       [Timedelta('1 days 00:00:00'), 0.15000152587890625],
       [Timedelta('1 days 00:00:00'), 0.019996643066399145],
       [Timedelta('1 days 00:00:00'), 0.17000198364258523]], dtype=object)

In [49]:
from statsmodels.tsa.ar_model import AutoReg

In [51]:
df_difference.isna().sum()

close    1
dtype: int64

In [52]:
res = AutoReg(df_difference.dropna(), lags = [1, 11, 12]).fit()

/home/matheus/miniconda3/envs/econometria/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [56]:
print(res.summary())

                            AutoReg Model Results                             
Dep. Variable:                  close   No. Observations:                 5676
Model:             Restr. AutoReg(12)   Log Likelihood               -4558.926
Method:               Conditional MLE   S.D. of innovations              0.541
Date:                Sun, 07 Aug 2022   AIC                           9127.851
Time:                        15:47:13   BIC                           9161.061
Sample:                            12   HQIC                          9139.417
                                 5676                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0052      0.007      0.723      0.469      -0.009       0.019
close.L1       0.0006      0.013      0.045      0.964      -0.025       0.027
close.L11      0.0083      0.013      0.627      0.5